In [76]:
import numpy as np

%load_ext autoreload
%autoreload 2

import xfmreadout.structures as structures

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [77]:
SHAPE=(400,20)
DIMENSIONS=(40,10)
MIN=0
MAX=1000
rng = np.random.default_rng()

dtype_list = [ np.float32, np.float64, np.int16, np.int32, np.int64, np.uint16, np.uint32, np.uint64 ]

for dtype_origin in dtype_list:

    if np.issubdtype(dtype_origin, np.floating):
        array_origin = rng.random(SHAPE, dtype=dtype_origin)*MAX
    else:
        array_origin = rng.integers(MIN, MAX, SHAPE, dtype=dtype_origin)

    for dtype_new in dtype_list:
        data = structures.DataSeries(array_origin, dimensions=DIMENSIONS)

        if np.issubdtype(dtype_new, np.floating):
            array_new = rng.random(SHAPE, dtype=dtype_new)*MAX
        else:
            array_new = rng.integers(MIN, MAX, SHAPE, dtype=dtype_new)

            data.assign(array_new)

        print(dtype_origin, dtype_new, data.check())
        
        assert data.check()




<class 'numpy.float32'> <class 'numpy.float32'> True
<class 'numpy.float32'> <class 'numpy.float64'> True
<class 'numpy.float32'> <class 'numpy.int16'> True
<class 'numpy.float32'> <class 'numpy.int32'> True
<class 'numpy.float32'> <class 'numpy.int64'> True
<class 'numpy.float32'> <class 'numpy.uint16'> True
<class 'numpy.float32'> <class 'numpy.uint32'> True
<class 'numpy.float32'> <class 'numpy.uint64'> True
<class 'numpy.float64'> <class 'numpy.float32'> True
<class 'numpy.float64'> <class 'numpy.float64'> True
<class 'numpy.float64'> <class 'numpy.int16'> True
<class 'numpy.float64'> <class 'numpy.int32'> True
<class 'numpy.float64'> <class 'numpy.int64'> True
<class 'numpy.float64'> <class 'numpy.uint16'> True
<class 'numpy.float64'> <class 'numpy.uint32'> True
<class 'numpy.float64'> <class 'numpy.uint64'> True
<class 'numpy.int16'> <class 'numpy.float32'> True
<class 'numpy.int16'> <class 'numpy.float64'> True
<class 'numpy.int16'> <class 'numpy.int16'> True
<class 'numpy.int16

In [84]:
import xfmreadout.utils as utils

SHAPE=(400,20)
DIMENSIONS=(40,10)
MIN=0
MAX=100
rng = np.random.default_rng()

dtype_list = [ np.float32, np.float64, np.int16, np.int32, np.int64, np.uint16, np.uint32, np.uint64 ]

for dtype_origin in dtype_list:
    if np.issubdtype(dtype_origin, np.floating):
        array_origin = rng.random(SHAPE, dtype=dtype_origin)*MAX
    else:
        array_origin = rng.integers(MIN, MAX, SHAPE, dtype=dtype_origin)

    for dtype_new in dtype_list:
        print(dtype_origin, dtype_new)
        array_new = utils.smartcast(array_origin, dtype_new)

        print(np.allclose(array_new, array_origin, atol=1))

<class 'numpy.float32'> <class 'numpy.float32'>
True
<class 'numpy.float32'> <class 'numpy.float64'>
True
<class 'numpy.float32'> <class 'numpy.int16'>
False
<class 'numpy.float32'> <class 'numpy.int32'>
False
<class 'numpy.float32'> <class 'numpy.int64'>
False
<class 'numpy.float32'> <class 'numpy.uint16'>
False
<class 'numpy.float32'> <class 'numpy.uint32'>
False
<class 'numpy.float32'> <class 'numpy.uint64'>
False
<class 'numpy.float64'> <class 'numpy.float32'>
True
<class 'numpy.float64'> <class 'numpy.float64'>
True
<class 'numpy.float64'> <class 'numpy.int16'>
False
<class 'numpy.float64'> <class 'numpy.int32'>
False
<class 'numpy.float64'> <class 'numpy.int64'>
False
<class 'numpy.float64'> <class 'numpy.uint16'>
False
<class 'numpy.float64'> <class 'numpy.uint32'>
False
<class 'numpy.float64'> <class 'numpy.uint64'>
False
<class 'numpy.int16'> <class 'numpy.float32'>
True
<class 'numpy.int16'> <class 'numpy.float64'>
True
<class 'numpy.int16'> <class 'numpy.int16'>
True
<class 

In [48]:
dtype_origin = np.float32
np.issubdtype(dtype_origin, np.floating)

True

In [29]:
nd = np.random.randint(0,100,(SHAPE))

data.assign(nd)

data.check()

TypeError: Cannot cast array data from dtype('int64') to dtype('float32') according to the rule 'safe'